<center>
    <img src="https://t3.ftcdn.net/jpg/03/32/11/34/360_F_332113482_OTYvz4irs36feOssRkNlIqyDo1oGW53k.jpg" width="20%" height="auto"/><br>
    <h1><b><font color=F39823>Data Sciencist</font><font color=5A595E> Case Study</font></b></h1><br>
    <img src="https://t3.ftcdn.net/jpg/03/32/11/34/360_F_332113482_OTYvz4irs36feOssRkNlIqyDo1oGW53k.jpg" width="20%" height="auto"/><br>
</center>
<br>

- **[<font color=#D65076>Generic Class Definitions</font>](#1GCD)**
    <br>[<font color=#5A595E>1. Model Preperation</font>](#1MP)
    <br>[<font color=#5A595E>2. Outlier</font>](#1O)
- **[<font color=#D65076>Data Exploration and Feature Engineering</font>](#2DEFE)**
    <br>[<font color=#5A595E>1. Users' Info</font>](#2UI)
    <br>[<font color=#5A595E>2. Transaction Data</font>](#2TD)
    <br>[<font color=#5A595E>3. Notification Data</font>](#2ND)
    <br>[<font color=#5A595E>4. Merge Data</font>](#2MD)
- **[<font color=#D65076>Model</font>](#3M)**
    <br>[<font color=#5A595E>1. Customer LifeTime Value (CLV)</font>](#3CLV)
    <br>[<font color=#5A595E>2. LightGBM</font>](#3L)

***
## <a id="1GCD"><font color=#D65076>1- Generic Class Definitions</font></a><br>
***

<p>In this phase, there are 2 classes are defined for general data analysis and outlier detection.</p>

The first class (model_preperation) is consisted of following features:

* **module checker (check_modules):** ensures if necessary modules are installed in the environment.
* **library importer (import_libraries) :** imports required libraries.
* **file reader (read_files):** reads or reloads all csv, excel and pickle files given path.
* **dataset describers (col_stats, df_cols):** describes dataset's information.
* **column summarizers (cat_sum, num_sum, col_sum):** gives specific informations about columns.

The second class (outliers) is consisted of following features:

* **check_outliers:** checks if there are outlier in numerical features.
* **grab_outliers:** returns dataframe with outlier values.
* **replace_with_thresholds:** returns outlier handled dataframe.

#### &#9654;<a id="1MP"><font color=#009B77> Model Preperation</font></a>

In [ ]:
class model_preperation:
    def __init__(self, display_=True, plot_=True, data='/kaggle/input', sep_=','):
        self.check_modules()
        self.import_libraries()
        self.df_list = []
        self.data = None
        self.read_files(data, sep_)
        self.display_ = display_
        self.plot_ = plot_

    def check_modules(self, modules={'pandas','numpy','matplotlib','seaborn','sklearn'}):
        print('Checking modules...')
        self.global_imports('sys')
        self.global_imports('subprocess')
        self.global_imports('pkg_resources')

        installed = {pkg.key for pkg in pkg_resources.working_set}
        missing = modules - installed

        if missing:
            self.install_modules(missing)

    def install_modules(self, modules):
        print('installing modules:'+' '.join(modules))
        python = sys.executable
        subprocess.check_call([python, '-m', 'pip', 'install', *modules], stdout=subprocess.DEVNULL)

    def global_imports(self, modulename, shortname = None):
        if shortname is None: 
            shortname = modulename

        print("\t","Import:",modulename,"as",shortname)
        if '.' not in modulename:
            globals()[shortname] = __import__(modulename)
        else:
            *modulename1, modulename2 = modulename.split('.')
            modulename1 = ".".join(modulename1)
            self.global_imports(modulename1)
            globals()[shortname] = eval(modulename1 + "." + modulename2)

    def import_libraries(self):
        print('Importing libraries...')
        
        self.global_imports('numpy','np')
        self.global_imports('pandas','pd')
        self.global_imports('os')
        self.global_imports('seaborn','sns')
        sns.axes_style("whitegrid")
        self.global_imports('matplotlib.pyplot','plt')
        self.global_imports('datetime.datetime','datetime')
        self.global_imports('functools.reduce','reduce')
        self.global_imports('warnings')
        warnings.simplefilter(action='ignore', category=FutureWarning)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', 50)
        pd.set_option('max_colwidth', 100)

    def read_files(self, data, sep_):
        print('Reading files...')
        if type(data)!=str:
            self.data = data
            exec(f"self.df_list.append('data')")
        else:
            ext_dict = {'csv':'csv','xlsx':'excel','pickle':'pickle'}
            for dir_name, _, file_names in os.walk(data):
                for file_name in file_names:
                    file = file_name.split('.')
                    file_ext = file[-1]
                    if  file_ext in ext_dict.keys():
                        if file_ext == 'xlsx':
                            self.check_modules({'openpyxl','xlrd'})
                        sep_str = f", sep='{sep_}'" if sep_!=',' else ''
                        command = f"self.{file[0]} = pd.read_{ext_dict[file_ext]}('{os.path.join(dir_name, file_name)}'"+sep_str+f")"
                        exec(command)
                        if file_ext != 'pickle':
                            exec(f"self.{file[0]} = self.reduce_mem_usage(self.{file[0]})")
                        exec(f"self.df_list.append('{file[0]}')")
                        print(f"\t {file[0]} is created.")
                        print()

    def save_dfs(self, path='/kaggle/output'):
        for d in self.df_list:
            full_path = os.path.join(path, d+'.pickle')
            exec(f"self.{d}.to_pickle('{full_path}')")
            print(f"{d} saved to {full_path}")

    def rename_dfs(self, rename_dict={}):
        self.global_imports('gc')
        for old, new in rename_dict.items():
            exec(f"self.{new} = self.{old}.copy()")
            exec(f"del self.{old}")
            exec("gc.collect()")
            exec(f"self.df_list.append('{new}')")
            exec(f"self.df_list.remove('{old}')")

    def set_dataframe(self, dataframe):
        if len(dataframe) == 0:
            dataframe = self.data.copy(deep=False) if self.data is not None else dataframe
        return dataframe

    def dtypes(self, dataframe=[], display_=None):
        dataframe = self.set_dataframe(dataframe)
        dtypes_df = pd.DataFrame(dataframe.dtypes).reset_index()
        dtypes_df.columns = ['COLUMN','DTYPE']
        dtypes_df['COLS'] = dtypes_df.astype(str).groupby(['DTYPE'])['COLUMN'].transform(lambda x: ', '.join(x))
        dtypes_df = dtypes_df[['DTYPE','COLS']].reset_index(drop=True).set_index('DTYPE').drop_duplicates()
        
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(dtypes_df)
            
        return dtypes_df
        
    def head_tail(self, dataframe=[], count=3, display_=None):
        dataframe = self.set_dataframe(dataframe)
        head_tail_df = pd.concat([dataframe.head(count),dataframe.tail(count)])
        
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(head_tail_df)
            
        return head_tail_df
    
    def drop_unnecessary_cols(self, dataframe=[], cols=[]):
        col_stats = self.col_stats(dataframe, display_=False);
        cols_to_drop_lst = col_stats.NUNIQUE[col_stats.NUNIQUE<=1].keys().to_list()
        return dataframe.drop(cols_to_drop_lst+cols, axis=1)

    def col_stats(self, dataframe=[], display_=None):
        dataframe = self.set_dataframe(dataframe)
        data_frames = [pd.DataFrame(dataframe.dtypes,columns=['DTYPES']),
                       pd.DataFrame(dataframe.isnull().sum(),columns=['IS_NULL']),
                       pd.DataFrame(dataframe.nunique(),columns=['NUNIQUE']),
                       dataframe.quantile([0, 0.01, 0.05, 0.50, 0.95, 0.99, 1]).T]
        col_stats_df = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True,how='outer'), data_frames)
        
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(col_stats_df)
            
        return col_stats_df

    def df_cols(self, dataframe=[], cat_th=10, car_th=20, id_th=100, display_=None):
        dataframe = self.set_dataframe(dataframe)

        numeric_cols   = dataframe._get_numeric_data().columns
        
        nunique        = dataframe.nunique()
        df_unique_pct  = nunique/dataframe.shape[0]
        self.id_cols   = list(set(list(df_unique_pct[(df_unique_pct>0.5)].keys()) + [col for col in list(nunique[nunique>id_th].keys()) if col[-2:].lower() == 'id']))
        
        self.cat_cols  = list(dataframe.select_dtypes(["category","object"]).columns)
        self.cat_but_car_cols = [col for col in self.cat_cols if (dataframe[col].nunique() > car_th) and (col not in self.id_cols)]
        self.num_but_cat_cols = [col for col in numeric_cols  if (dataframe[col].nunique() < cat_th) and (col not in self.id_cols)]
        self.num_but_cat_but_car_cols = [col for col in self.num_but_cat_cols if (dataframe[col].nunique() > car_th) and (col not in self.id_cols)]
        self.num_but_cat_cols = [col for col in self.num_but_cat_cols  if col not in self.num_but_cat_but_car_cols]
        
        self.cat_cols  = [col for col in self.cat_cols if col not in self.cat_but_car_cols+self.id_cols]
        self.num_cols  = [col for col in numeric_cols  if col not in self.num_but_cat_cols+self.num_but_cat_but_car_cols+self.id_cols]

        self.date_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "datetime64[ns]"]        
        self.id_cols   = [col for col in self.id_cols if col not in self.date_cols]
        

        stats = pd.DataFrame(index=['ID Cols', 'Date Cols', 'Numeric Cols', 'Categoric (Low Cardinality) Cols', 
                                    'Categoric (High Cardinality) Cols', 'Numeric But Categoric (Low Cardinality) Cols',  
                                    'Numeric But Categoric (High Cardinality) Cols'])
        stats['Count'] = [len(self.id_cols), len(self.date_cols), len(self.num_cols), len(self.cat_cols), \
                          len(self.cat_but_car_cols), len(self.num_but_cat_cols), len(self.num_but_cat_but_car_cols)]
        stats['Columns'] = [", ".join(self.id_cols), ", ".join(self.date_cols), ", ".join(self.num_cols), ", ".join(self.cat_cols), \
                            ", ".join(self.cat_but_car_cols), ", ".join(self.num_but_cat_cols), ", ".join(self.num_but_cat_but_car_cols)]
        
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(stats)
        
        return self.id_cols, self.date_cols, self.num_cols, self.cat_cols, self.cat_but_car_cols, self.num_but_cat_cols, self.num_but_cat_but_car_cols

    def plot_cols(self, dataframe=[], col_name=None, kind='bar', palette='Set2'):
        dataframe = self.set_dataframe(dataframe)
        plt.figure(figsize=(8,5))
        
        if kind == 'hist':
            sns.set_palette(sns.color_palette(palette))
            ax = sns.histplot(dataframe[col_name], bins=20, kde=True)
            plt.xlabel('')
        if kind == 'bar':
            sns.set_palette(sns.color_palette(palette))
            dataframe.index.names = ['index']
            ax = sns.barplot(x='index', y=col_name, data=dataframe.reset_index())
            ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
            plt.xlabel('')

        plt.title(col_name)
        plt.tight_layout()
        plt.show()

    def cat_summ(self, dataframe=[], col_name=None, threshold=False, display_=None, plot_=None):
        dataframe = self.set_dataframe(dataframe)
        df = pd.DataFrame({col_name: dataframe[col_name].value_counts(),"Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)})
        
        if threshold!=False:
            threshold_column = col_name if threshold > 1 else 'Ratio'
            if threshold <=1:
                  df[threshold_column] = df[threshold_column]/100 
            main_df = df[df[threshold_column]>threshold]
            th_df = pd.DataFrame(df[df[threshold_column]<threshold].sum()).T
            th_df.index = ['Others']
            cat_summary_df = pd.concat([main_df,th_df])
        else:
            cat_summary_df = df
        
        plot_ = self.plot_ if plot_ is None else plot_
        if plot_:
            self.plot_cols(cat_summary_df,col_name)
        
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(cat_summary_df)
            
        return cat_summary_df

    def num_summ(self, dataframe=[], col_name=None, display_=None, plot_=None,
                   quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]):

        dataframe = self.set_dataframe(dataframe)
        num_summary_df = pd.DataFrame(dataframe[col_name].describe(quantiles).T)
        
        plot_ = self.plot_ if plot_ is None else plot_
        if plot_:
            self.plot_cols(dataframe,col_name,'hist',"rainbow_r")
            
        if display_:
            display(num_summary_df)
            
        return num_summary_df
        
    def col_summ(self, dataframe=[], target=None, col_name=None, display_=None, plot_=None):
        dataframe = self.set_dataframe(dataframe)
        cat_cols, num_cols, cat_but_car_cols, num_but_cat_cols, date_cols = self.df_cols(dataframe)
        
        if col_name in cat_cols+num_but_cat_cols:
            col_target_summary_df = pd.DataFrame({col_name: dataframe.groupby(col_name)[target].mean()})
        elif col_name in num_cols+cat_but_car_cols:
            col_target_summary_df = dataframe.groupby(target).agg({col_name: "mean"}).sort_values(by=col_name,ascending=False)
        
        plot_ = self.plot_ if plot_ is None else plot_
        if plot_:
            self.plot_cols(col_target_summary_df,col_name)
            
        display_ = self.display_ if display_ is None else display_
        if display_:
            display(col_target_summary_df)
            
        return col_target_summary_df
    
    def reduce_mem_usage(self, df):
        start_mem = df.memory_usage().sum() / 1024**2
        print('\t Memory usage of dataframe is {:.2f} MB'.format(start_mem))
        for col in df.columns:
            col_type = df[col].dtype
            if col_type == object:
                if df[col].nunique()<300:
                    df[col] = df[col].astype('category')
                    continue
            if str(col_type)[:3] in ['int','flo']:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        
        end_mem = df.memory_usage().sum() / 1024**2
        print('\t Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('\t Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
            
        return df

#### &#9654;<a id="1O"><font color=#009B77> Outlier</font></a>

In [ ]:
class Outlier:
    
    def __init__(self, dataframe, column, low_quantile=0.05, up_quantile=0.95):
        self.dataframe = dataframe
        self.column = column
        self.low_quantile = low_quantile
        self.up_quantile = up_quantile
        self.calculate_data_will_be_affected()
        
  
    def calculate_data_will_be_affected(self):
        if self.check_outlier():
            ratio_for_up_limit = (len(self.dataframe[self.dataframe[self.column] > self.dataframe[self.column].
                                      quantile(self.up_quantile)]) /
                                  self.dataframe[self.column].shape[0]) * 100
            ratio_for_low_limit = (len(self.dataframe[self.dataframe[self.column] < self.dataframe[self.column].
                                   quantile(self.low_quantile)]) /
                                   self.dataframe[self.column].shape[0]) * 100
            ratio_of_affected_data = round(ratio_for_up_limit + ratio_for_low_limit, 2)
            print("{} - When we reassign operation in the ({} - {}) range, %{} of data "
                  "will be affected in this operation.".format(self.column,
                                                               self.low_quantile,
                                                               self.up_quantile,
                                                               ratio_of_affected_data))
            return ratio_for_up_limit, ratio_for_low_limit, ratio_of_affected_data
        else:
            print("{} - When we reassign operation in the ({} - {}) range, no data "
                  "will be affected in this operation.".format(self.column,
                                                               self.low_quantile,
                                                               self.up_quantile))
    
    def outlier_thresholds(self):
        quartile1 = self.dataframe[self.column].quantile(self.low_quantile)
        quartile3 = self.dataframe[self.column].quantile(self.up_quantile)
        interquantile_range = quartile3 - quartile1
        up_limit = quartile3 + 1.5 * interquantile_range
        low_limit = quartile1 - 1.5 * interquantile_range
        return low_limit, up_limit
    
    def grab_outliers(self, index=False):
        low, up = self.outlier_thresholds()
        if self.dataframe[((self.dataframe[self.column] < low) |
                      (self.dataframe[self.column] > up))].shape[0] > 10:
            return self.dataframe[((self.dataframe[self.column] < low) |
                             (self.dataframe[self.column] > up))]
        else:
            return self.dataframe[((self.dataframe[self.column] < low) |
                             (self.dataframe[self.column] > up))]

        if index:
            outlier_index = self.dataframe[((self.dataframe[self.column] < low) |
                                       (self.dataframe[self.column] > up))].index
            return outlier_index
    
    def check_outlier(self):
        low_limit, up_limit = self.outlier_thresholds()
        if self.dataframe[(self.dataframe[self.column] > up_limit) |
                     (self.dataframe[self.column] < low_limit)].any(axis=None):
            return True
        else:
            return False
    
    def replace_with_thresholds(self):
        low_limit, up_limit = self.outlier_thresholds()
        if low_limit > 0:
            self.dataframe.loc[(self.dataframe[self.column] < low_limit), self.column] = low_limit
            self.dataframe.loc[(self.dataframe[self.column] > up_limit), self.column] = up_limit
        else:
            self.dataframe.loc[(self.dataframe[self.column] > up_limit), self.column] = up_limit
        return self.dataframe

### <font color=#009B77>Initial Call for the Model Preperation Class</font>

<b>Note:</b> This field should be run if the pickle files have not been created before. Otherwise [next cell](#next_cell) can be run to save some time.

In [ ]:
files_path='../input/case-study/Data_Science_case_study/data/'
mp = model_preperation(data=files_path, plot_=True, display_=True, sep_='|')
mp.rename_dfs({'notifications_dataset':'ntf_data','transactions_dataset':'trx_data','users_dataset':'user_data'})
mp.save_dfs('./')

### <a id='next_cell'><font color=#009B77>Call for the Model Preperation Class</font></a>

<p><b>Note:</b> The cell below should be run after the pickle files are created. (can be run repeatedely)</p>

In [ ]:
mp = model_preperation(data='./', plot_=False, display_=False)

***
## <a id="2DEFE"><font color=#D65076>2- Data Exploration and Feature Engineering</font></a><br>
***

#### &#9654;<a id="2UI"><font color=#009B77> Users' Info</font></a>

In [ ]:
mp.head_tail(mp.user_data);
col_stats = mp.col_stats(mp.user_data)

In [ ]:
# Let's cast Activation_Date and Crypto_Curr_Flag
mp.user_data.Activation_Date  = pd.to_datetime(mp.user_data.Activation_Date)
mp.user_data.Crypto_Curr_Flag = mp.user_data.Crypto_Curr_Flag.astype(str)

# Create Tenure column
mp.trx_data.Transaction_Date = pd.to_datetime(mp.trx_data.Transaction_Date)
now = mp.trx_data.Transaction_Date.max()
mp.user_data.loc[:,'_Tenure_in_Days'] = mp.user_data.Activation_Date.apply(lambda x:(now-x).days).astype(np.int16)

# Check corr of notification flags
acc = np.sum(np.equal(mp.user_data.Email_Notif_Allowed, mp.user_data.Push_Notif_Allowed ))/len(mp.user_data.Email_Notif_Allowed)
print(f'Correlation of email and push notification approval is {100*acc:.2f}%')

# Create Notification Column
mp.user_data.loc[:,'_Notif_Allowed'] = (mp.user_data.Email_Notif_Allowed.astype(str) + mp.user_data.Push_Notif_Allowed.astype(str))
mp.user_data.loc[:,'_Notif_Allowed'] = mp.user_data.loc[:,'_Notif_Allowed'].map({'00':'0', '01':'1', '10':'1', '11':'1', 'UnknownUnknown':'Unknown'}).astype('category')

# Drop unnecessary columns
mp.user_data = mp.drop_unnecessary_cols(mp.user_data, ['Activation_Date','Email_Notif_Allowed','Push_Notif_Allowed'])

col_stats = mp.col_stats(mp.user_data)
id_cols, date_cols, num_cols, cat_cols, cat_but_car_cols, num_but_cat_cols, num_but_cat_but_car_cols = mp.df_cols(mp.user_data, display_=True, cat_th=32, car_th=10)

In [ ]:
# Check if there are outliers for num_cols.
for col in num_cols:
    outlier = Outlier(mp.user_data, col, 0.05, 0.95)
    display(outlier.grab_outliers()[col].unique())
    print()

In [ ]:
# Plot all columns' summary
for cat_col in cat_but_car_cols:
    mp.cat_summ(mp.user_data, cat_col, threshold=0.01);
for cat_col in cat_cols:
    mp.cat_summ(mp.user_data, cat_col, threshold=False);
# Analyze num_cols
for num_col in num_cols:
    mp.num_summ(mp.user_data, num_col)

#### &#9654;<a id="2TD"><font color=#009B77> Transaction Data</font></a>

In [ ]:
mp.head_tail(mp.trx_data);
col_stats = mp.col_stats(mp.trx_data)

In [ ]:
# Let's cast Year_Month
mp.trx_data.Year_Month = mp.trx_data.Year_Month.astype(str)

# Fill null values with "UNKNOWN"
for col in col_stats[col_stats.IS_NULL>0].index:
    try:
        mp.trx_data.loc[:,col] = mp.trx_data[col].fillna('UNKNOWN')    
    except:
        mp.trx_data.loc[:,col] = mp.trx_data[col].cat.add_categories("UNKNOWN").fillna('UNKNOWN')

col_stats = mp.col_stats(mp.trx_data)
id_cols, date_cols, num_cols, cat_cols, cat_but_car_cols, num_but_cat_cols, num_but_cat_but_car_cols = mp.df_cols(mp.trx_data, display_=True, cat_th=32, car_th=10, id_th=1000)
# DataFrame[ColName+"_Imputed"] =   np.where(DataFrame[ColName].isnull(),1,0)

In [ ]:
# Check if there are outliers for num_cols.
for col in num_cols:
    outlier = Outlier(mp.trx_data, col, 0.05, 0.95)
    display(outlier.grab_outliers()[col].max())
    print()

In [ ]:
# Plot all columns' summary
for cat_col in cat_but_car_cols:
    mp.cat_summ(mp.trx_data, cat_col, threshold=0.01);
for cat_col in cat_cols:
    mp.cat_summ(mp.trx_data, cat_col, threshold=False);
# Analyze num_cols
for num_col in num_cols:
    mp.num_summ(mp.trx_data, num_col)

#### &#9654;<a id="2ND"><font color=#009B77> Notification Data</font></a>

In [ ]:
mp.head_tail(mp.ntf_data);
col_stats = mp.col_stats(mp.ntf_data)

In [ ]:
# Let's cast Process_Date
mp.ntf_data.Process_Date = pd.to_datetime(mp.ntf_data.Process_Date)

# Drop unnecessary columns
mp.ntf_data = mp.drop_unnecessary_cols(mp.ntf_data, ['Unnamed: 0'])

col_stats = mp.col_stats(mp.ntf_data)
id_cols, date_cols, num_cols, cat_cols, cat_but_car_cols, num_but_cat_cols, num_but_cat_but_car_cols = mp.df_cols(mp.ntf_data, display_=True, cat_th=32, car_th=10, id_th=1000)

In [ ]:
# Plot all columns' summary
for cat_col in cat_but_car_cols:
    mp.cat_summ(mp.ntf_data, cat_col, threshold=0.01);
for cat_col in cat_cols:
    mp.cat_summ(mp.ntf_data, cat_col, threshold=False);
# Analyze num_cols
for num_col in num_cols:
    mp.num_summ(mp.ntf_data, num_col)

In [ ]:
channel_comm_reason    = mp.ntf_data.groupby(['Communication_Reason','Channel'])['Customer_ID'].count()
channel_comm_reason_df = pd.DataFrame(channel_comm_reason)
channel_comm_reason_df = channel_comm_reason_df[channel_comm_reason_df['Customer_ID']!=0].reset_index()
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.tight_layout()
ax = sns.barplot(x="Communication_Reason", hue="Channel", y="Customer_ID", data=channel_comm_reason_df, palette='turbo', ax=axes[0]);
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
sns.heatmap(channel_comm_reason.unstack(), cmap='GnBu', square=True, robust=True, ax=axes[1]);

#### &#9654;<a id="2MD"><font color=#009B77> Merge Data</font></a>

<p>Next mission is merging all datasets by <b>Customer_ID</b>. Before moving on to this, <u>some aggregations/calculations have to be made.</u></p>

<p><b>For the notification dataset:</b> <font color=#DD4124>Notification Data Channel</font> and <font color=#DD4124>Communication Reasons</font>' count can be pivoted per customer as follows:</p>
    
Note that, the aggregations are done in 2 phases, including all data and 2021 only. 

In [ ]:
## Stacking process
stacked = mp.ntf_data.groupby(['Customer_ID','Communication_Reason','Channel'])[['Customer_ID']].count()
stacked.columns = ['Sum']
stacked = stacked.reset_index().pivot(index='Customer_ID',columns=['Communication_Reason','Channel'], values='Sum')
stacked.columns = ['_'+x for x in stacked.columns.map('_'.join)]

## Same process for only 2021 data
stacked2021 = mp.ntf_data[mp.ntf_data.Process_Date.dt.year == 2021].groupby(['Customer_ID','Communication_Reason','Channel'])[['Customer_ID']].count()
stacked2021.columns = ['Sum']
stacked2021 = stacked2021.reset_index().pivot(index='Customer_ID',columns=['Communication_Reason','Channel'], values='Sum')
stacked2021.columns = ['_'+x+'_2021' for x in stacked2021.columns.map('_'.join)]

## Drop unnecessary columns
drop_list = []
for col in stacked2021.columns:
    if sum(stacked2021[col].value_counts()/stacked2021.shape[0]>0.05)<2:
        drop_list.append(col)

stacked = mp.drop_unnecessary_cols(stacked)
stacked2021 = mp.drop_unnecessary_cols(stacked2021,drop_list)

In [ ]:
mp.user_data_copy = mp.user_data.copy()

In [ ]:
# Let's merge notifications_data,  user_data, transaction_data
mp.user_data=mp.user_data_copy.copy()
mp.user_data = pd.merge(pd.merge(mp.user_data, stacked, right_index=True, how='left', left_on='Customer_ID'), stacked2021, right_index=True, how='left', left_on='Customer_ID')
data = pd.merge(mp.user_data,mp.trx_data,on='Customer_ID')
mp.user_data = mp.reduce_mem_usage(mp.user_data)

In [ ]:
transaction_amount = mp.trx_data.groupby(['Customer_ID'])['Transaction_Amount_Usd'].agg(['sum','min','max','mean','std'])
transaction_amount_2021 = mp.trx_data[mp.trx_data['Year_Month'].astype(int)>=202101].groupby(['Customer_ID'])['Transaction_Amount_Usd'].agg(['sum','min','max','mean','std'])
transaction_amount.columns = ['_Transaction_Amount_Usd_'+x for x in transaction_amount.columns]
transaction_amount_2021.columns = ['_Transaction_Amount_Usd_2021_'+x for x in transaction_amount_2021.columns]

transaction_type = mp.trx_data.groupby(['Customer_ID','Transaction_Type','Direction','Transaction_Status'])[['Transaction_Type']].count()
transaction_type.columns = ['Count']
transaction_type = transaction_type.reset_index().pivot(index='Customer_ID',columns=['Transaction_Type','Direction','Transaction_Status'], values='Count')
transaction_type.columns = ['_'+x for x in transaction_type.columns.map('_'.join)]
transaction_type = mp.drop_unnecessary_cols(transaction_type)

Cardholder = mp.trx_data.groupby(['Customer_ID','Cardholder_Verified'])[['Customer_ID']].count()
Cardholder.columns = ['Count']
Cardholder = Cardholder.reset_index().pivot(index='Customer_ID',columns=['Cardholder_Verified'], values='Count')
Cardholder.columns = ['_Cardholder_'+x for x in Cardholder.columns]

Currency_Code_First = mp.trx_data.groupby('Customer_ID')[['Currency_Code']].agg('first')
Currency_Code_First.columns = ['_'+x+'_first' for x in Currency_Code_First.columns]
Currency_Code_Last  = mp.trx_data.groupby('Customer_ID')[['Currency_Code']].agg('last')
Currency_Code_Last.columns = ['_'+x+'_last' for x in Currency_Code_Last.columns]

Merchant_Country_First = mp.trx_data.groupby('Customer_ID')[['Merchant_Country']].agg('first')
Merchant_Country_First.columns = ['_'+x+'_first' for x in Merchant_Country_First.columns]
Merchant_Country_Last  = mp.trx_data.groupby('Customer_ID')[['Merchant_Country']].agg('last')
Merchant_Country_Last.columns = ['_'+x+'_last' for x in Merchant_Country_Last.columns]

Merchant_MCC_Code_First  = mp.trx_data.groupby('Customer_ID')[['Merchant_MCC_Code']].agg('first')
Merchant_MCC_Code_First.columns = ['_'+x+'_first' for x in Merchant_MCC_Code_First.columns]
Merchant_MCC_Code_Last   = mp.trx_data.groupby('Customer_ID')[['Merchant_MCC_Code']].agg('last')
Merchant_MCC_Code_Last.columns = ['_'+x+'_last' for x in Merchant_MCC_Code_Last.columns]

mp.user_data = pd.merge(mp.user_data, transaction_amount, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, transaction_amount_2021, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, transaction_type, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Cardholder, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Currency_Code_First, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Currency_Code_Last, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Merchant_Country_First, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Merchant_Country_Last, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Merchant_MCC_Code_First, right_index=True, how='left', left_on='Customer_ID')
mp.user_data = pd.merge(mp.user_data, Merchant_MCC_Code_Last, right_index=True, how='left', left_on='Customer_ID')

In [ ]:
mp.user_data = mp.reduce_mem_usage(mp.user_data)
col_stats = mp.col_stats(mp.user_data)
for col in col_stats[col_stats.IS_NULL!=0].index:
    if col not in ["_Currency_Code_first","_Currency_Code_last","_Merchant_Country_first","_Merchant_Country_last","_Merchant_MCC_Code_first","_Merchant_MCC_Code_last"]:
        mp.user_data[col] = mp.user_data[col].fillna(0)

***
## <a id="3M"><font color=#D65076>3- Model</font></a><br>
***

#### &#9654;<a id="3CLV"><font color=#009B77> Customer LifeTime Value (CLV)</font></a>

In [ ]:
outlier = Outlier(data, 'Transaction_Amount_Usd', 0.01, 0.99)
display(outlier.grab_outliers()['Transaction_Amount_Usd'].unique())
data = outlier.replace_with_thresholds()

In [ ]:
df = data.loc[data['Transaction_Status']=='COMPLETED',['Transaction_ID','Customer_ID','Transaction_Date','Transaction_Amount_Usd']]

now = df.Transaction_Date.max()
rfm = df.groupby('Customer_ID').agg({'Transaction_Date': [lambda date: (max(now-date.min(),date.max()-date.min())).days,
                                                          lambda date: (now-date.max()).days],
                                     'Transaction_ID': lambda num: num.nunique(),
                                     'Transaction_Amount_Usd': lambda TotalPrice: TotalPrice.sum()})

rfm.columns = rfm.columns.droplevel(0)
rfm.columns=['tenure','recency','frequency','monetary']
rfm.monetary = rfm.monetary / rfm.frequency
rfm.loc[:,'recency_weekly'] = rfm.recency / 7
rfm.loc[:,'tenure_weekly'] = rfm.tenure / 7

rfm = rfm[(rfm['frequency'] > 1)]
rfm[rfm.recency>rfm.tenure]

In [ ]:
_ = !pip install lifetimes;
from lifetimes import BetaGeoFitter, GammaGammaFitter

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(rfm['frequency'],
        rfm['recency_weekly'],
        rfm['tenure_weekly'])

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(rfm['frequency'], rfm['monetary']);

cltv = ggf.customer_lifetime_value(bgf,
                                   rfm['frequency'],
                                   rfm['recency_weekly'],
                                   rfm['tenure_weekly'],
                                   rfm['monetary'],
                                   time=3,
                                   freq="W",
                                   discount_rate=0.01)

pd.set_option('display.float_format', lambda x: '%.5f' % x)
cltv = pd.DataFrame(cltv).sort_values('clv',ascending=False)

cltv[cltv.clv==0].count()

In [ ]:
cltv['Churn']= np.where(cltv.clv==0,1,0)
cltvrfm = pd.merge(cltv,rfm,left_index=True,right_on='Customer_ID')

In [ ]:
cltvrfm = cltvrfm[['Churn']].reset_index()
cltvrfm.to_csv('clv.csv')

#### &#9654;<a id="3L"><font color=#009B77> LightGBM</font></a>

In [ ]:
data = mp.user_data.copy()
target_col = 'Is_Engaged'

In [ ]:
id_cols, date_cols, num_cols, cat_cols, cat_but_cr_cols, num_but_cat_cols, num_but_cat_but_car_cols = mp.df_cols(data, display_=True, cat_th=32, car_th=10, id_th=1000)

"""
engaged_id_array = mp.trx_data[(mp.trx_data.Transaction_Status=='COMPLETED') & (mp.trx_data.Year_Month.astype(int)>=202101)]\
                    .groupby(['Customer_ID','Year_Month'])[['Transaction_ID']].count().reset_index().Customer_ID.unique()

mp.user_data['Is_Engaged'] = np.where(mp.user_data.Customer_ID.isin(engaged_id_array),1,0)
""";
data = pd.merge(data,cltvrfm, on='Customer_ID',how='left')
data['Is_Engaged'] = 1-data['Churn']
data.drop('Churn',axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import *
import lightgbm as lgb
from category_encoders.hashing import HashingEncoder

In [ ]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols,
                               drop_first=drop_first)
    return dataframe

def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() /
                        len(temp_df) < rare_perc).any(axis=None)]
    print(rare_columns)

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare',
                                temp_df[var])
    return temp_df

def drop_rare_elements(dataframe, column_name='', drop_text='Others', threshold=0.05):
    total = dataframe[column_name].shape[0]
    percentage = dataframe[column_name].value_counts() / total
    rare_elements_list = []
    for k,v in percentage.items():
        if v < threshold:
            rare_elements_list.append(k)
    dataframe.loc[dataframe[column_name].isin(rare_elements_list),column_name] = drop_text
    return dataframe

data = one_hot_encoder(data,[x for x in cat_cols+num_but_cat_cols if x != target_col],drop_first=True)
data = mp.drop_unnecessary_cols(data)

In [ ]:
data_na = data[data[target_col].isna()]
data = data[data[target_col].notna()]

X = data.drop([x for x in data.columns if (x in ['Customer_ID',target_col])],axis=1)
y = data[target_col].astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [ ]:
params={
         'learning_rate': 0.014,
         'max_depth': 8,
         'n_estimators': 2000
}

class_weights = dict(zip([0,1],(y_train.value_counts()/y_train.shape[0]*5).values))

lgbm_model = lgb.LGBMClassifier(**params,is_unbalance=True,class_weight=class_weights).fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
y_pred = lgbm_model.predict(X_test)                                                                                  


print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('LightGBM Model balanced accuracy score: {0:0.4f}'.format(balanced_accuracy_score(y_test, y_pred)))
print('LightGBM Model precision score: {0:0.4f}'.format(precision_score(y_test, y_pred)))
print('LightGBM Model recall score: {0:0.4f}'.format(recall_score(y_test, y_pred)))
print('LightGBM Model f1 score: {0:0.4f}'.format(f1_score(y_test, y_pred)))
print('LightGBM Model fbeta score: {0:0.4f}'.format(fbeta_score(y_test, y_pred, 0.5)))
print('LightGBM Model roc auc score: {0:0.4f}'.format(roc_auc_score(y_test, y_pred)))
print('LightGBM Model cohen kappa score: {0:0.4f}'.format(cohen_kappa_score(y_test, y_pred)))
print('LightGBM Model matthews correlation coefficient score: {0:0.4f}'.format(matthews_corrcoef(y_test, y_pred)))

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(cm)
print('LightGBM Model false positive rate: {0:0.4f}'.format(fp / (fp + tn)))
print('LightGBM Model false negative rate: {0:0.4f}'.format( fn / (tp + fn)))

In [ ]:
def plotImp(model, X , num = 20, fig_size = (20, 10)):
    feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 2)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()
    
plotImp(lgbm_model, X_train)